In [ ]:
'''
author: Zhexuan Gu
Date: 2022-11-26 15:24:03
LastEditTime: 2022-11-26 18:53:08
FilePath: /Assignment2/utils/MyResnet.py
Description: Please implement
'''
import torch
import torch.nn as nn
import torch.nn.functional as F

# emulate source code in torchvision resnet.py，but I won't handle so many exceptions 
# and I won't build a very generalized model

class BasicBlock(nn.Module):
    """
        There're two modes:
            when stride is 1, we keep the same shape;
            when stride is 2, since the feature map H W is halved! we double the channels (num of feature map)!
    """
    def __init__(self, in_channels:int, out_channels:int, stride=1) -> None:
        super().__init__()
        """
        this basic structure is just  the key idea of the paper
        let the input go across to convs and there's another short cut to directly add the input
        """
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        """
        use batchnormalization can help make model easier to train and ease the problem of gradient
        """
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.re1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.re2 = nn.ReLU()
        
        """
        then is the tricky part:
            we directly add a short-cut
            but what we should focus is:  Is the shape of the input data same as conv output?
            it determines whether we can directly add them together
        """
        self.shortcut = nn.Sequential()   # first we set it as nothing, this situation corresponds to the same shape
        
        """
        then it's important about how to judge whether the shape is same, recall the conv output shape formula:
            H' = 1 + (H + 2 * pad - HH) / stride                 W' = 1 + (W + 2 * pad - WW) / stride 
            
        since pad=1, it's obvious that if stride=1, then we can keep the shape as same as the input data
        another thing is about the channels, we should make sure number of channels do not change
        """
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
            """
            here, the idea is we try to use 1*1 conv layer to change the output channels and downsample the size
            
                consider:
                    if out input is (28, 28), but we set stride = 2
                    for the 2 conv layers:
                        H1 = W1 = (28 + 1 * 2 - 3) // 2 + 1 = 14
                        H2 = W2 = (14 + 1 * 2 - 3) // 1 + 1 = 14
                    
                    for the shortcut:
                        H' = W' = (28 + 0[no padding] - 1) // 2 + 1 = 14
                Therefore, shape is kept as same
            """
    
    def forward(self, x):
        """
        network feed forward, it's very simple we just need to give the input then it'll be ok
        """
        out1 = self.re1(self.bn1(self.conv1(x)))
        out2 = self.bn2(self.conv2(out1))
        out = self.re2(out2 + self.shortcut(x))

        # out = self.re2(out2)
        return out
    
    """
    -----------------with the above code-----------------
    we 've implemented the most significant idea
    """
    

"""
As the official document says, BottleNeck is a variant of original ResNet, 
so I'm not intend to finish it now!
"""

class ResNet(nn.Module):
    def __init__(self, block, num_blocks:list, num_classes=10) -> None:
        super().__init__()
        
        # since the input is a color image, the channels are 3.
        # according to the paper, we set the fisrt conv layer
        self.out_channels = 64
        self.conv1 = nn.Conv2d(3, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.out_channels)
        self.re1 = nn.ReLU()
        
        """
        we can check the size of images in cifar-10 dataset is (32, 32)
            therefore, after the first layer, the ouput is 
            (32 + 1 * 2 - 3) // 1 + 1 = 32
                (batch_size, 64, 32, 32)
        """
        """
        the rest architecture is all about our Basic residuals, so we create a function "make_layer" to 
        implement it
            block: our defined class that can help us initialize a basic residual
            channel: input data channel to the block
            outchannel: as we have mentioned in BasicResidual Class, we determine this value by stride!
            num_blocks: no other meaning, just block number
            stride: just stride to control the residual block
        """
        self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
        # since the stride becomes 2, double our output channels
        self.layer2 = self.make_layer(block, 128, num_blocks[1], stride=2)
        # double again
        self.layer3 = self.make_layer(block, 256, num_blocks[2], stride=2)
        
        self.layer4 = self.make_layer(block, 512, num_blocks[2], stride=2)
        
        """
        finally it's a fully connected layer, ouput the scores of each class
        """
        self.fc1 = nn.Linear(512, num_classes)
    
    
    def make_layer(self, block:BasicBlock, channels:int, num_blocks:int, stride):
        """
            so, actually, the parameters are just what we have already mentioned in to BasicBlock Class.
        """
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.out_channels, channels, stride))
            # since sometimes we double the output channels, next time the input should change
            self.out_channels = channels
            # ...
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out1 = self.re1(self.bn1(self.conv1(x)))
        out2 = self.layer1(out1)
        out3 = self.layer2(out2)
        out4 = self.layer3(out3)
        out5 = self.layer4(out4)

        out5 = F.avg_pool2d(out5, 4)
        """
        till now, out5 is still 4 dimensional
        to make use of Linear layer, it's important to make it a two dimensional vector!
        
        the second parameter 1 means it starts squeeze the 4 dimensional thing from second dimension
        so it'll keep the batch size
        """
        out5_flat = torch.flatten(out5, 1)
        out = self.fc1(out5_flat)
        return out
    


def ResNet18():
    """
    to create a 18 layers Resnet model:

        so a nice choice of block_nums list is [2, 2, 2, 2]
        since one basic block contains two conv layers:
            the total num of network is:

                1 (fixed conv layer) + 2 * 2 * 4 (residual layers) + 1 (fc layer) = 18
    """
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
import torch 
import torch.nn as nn
import torch.optim as opt
from torch.utils.data import sampler
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
dtype = torch.float32

def check_accuracy(loader:DataLoader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    acc = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc


def train(loader:DataLoader, model, epochs:int, loader_val:DataLoader, loss_list:list, acc_list:list):
    best_acc = 0.0

    # other hyperparameters setting 
    optimizer = opt.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)    # Adam optimizer instead of SGD                       
    loss = None
    model = model.to(device=device)
    for epoch in range(epochs):
        print("---------------epoch %d--------------" % (epoch + 1))

        for t, (x, y) in enumerate(loader):

            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            model.train(True)
            
            preds = model(x)
            loss = F.cross_entropy(preds, y)
            
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            
            if t % 100 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                acc = check_accuracy(loader_val, model)
                print()

                if acc > best_acc:
                  best_acc = acc
                  # save the model
                  torch.save(model, "./best_model_plainnet.pt")
        loss_list.append(loss.item())
        acc_list.append(acc)



if __name__ == "__main__":
    # data augmentation
    transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
])

    transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


    # since we don't have the dataset, it's important to download it first
    train_dataset = torchvision.datasets.CIFAR10(root="./dataset", 
                                                 transform=transform_train,
                                                 train=True, 
                                                 download=True)
    test_dataset = torchvision.datasets.CIFAR10(root="./dataset", 
                                                transform=transform_test,
                                                train=False, 
                                                download=True)
    
    # create dataloader so that we can train a batch of images simutaneously
    NUM_TRAIN = 49000
    train_loader = DataLoader(train_dataset,
                              sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)),
                              batch_size=128,
                              num_workers=2)
    val_loader = DataLoader(train_dataset,
                              sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)),
                              batch_size=64)
    test_loader = DataLoader(test_dataset, 
                             batch_size=100,
                             num_workers=2)
    
    model = ResNet18()
    loss_list, acc_list = [], []
    train(train_loader, model, 50, val_loader, loss_list, acc_list)

# 新段落

In [ ]:
model = torch.load('./best_model.pt')
check_accuracy(test_loader, model)



In [ ]:
import matplotlib.pyplot as plt

epoch = [i for i in range(50)]
plt.plot(epoch, acc_list)
plt.xlabel("epoch")
plt.ylabel("accuracy on validation set")
plt.show()

In [ ]:
plt.plot(epoch, loss_list)
plt.xlabel("epoch")
plt.ylabel("loss on training set")
plt.show()